# Lab 6 - Populating and Managing Match-action Tables at Runtime

<div style="text-align: justify;">
This lab will walk you through creating an experiment that contains a P4-DPDK programmable pipeline. It also describes how to populate and manage match-action tables at runtime. It then explains a tool (DPDK pipeline CLI) that is used to execute DPDK commands at runtime.
<figure style="text-align: center;">
<img src="./labs_files/lab6/figs/0_00_fabric_topology.png" width="700px"><br>

# Introduction

## Runtime
<div style="text-align: justify;">
Once a P4 program is compiled into a target-specific configuration, the output is loaded into the data plane of the device. Then, the behavior of the P4 target can be managed at runtime by the control plane via data plane Application Programming Interface (APIs). Runtime operations include inserting, updating, and deleting entries in P4 tables as well as controlling other entities of the program also called externs such as registers, counters, meters, etc. </div> <br>
<div style="text-align: justify;">
Runtime APIs can be divided into program-dependent and program-independent APIs. Program-dependent APIs comprise functions whose names are derived from the P4 program itself. Thus, any changes to the P4 program would modify the names and the definitions of the APIs’ functions. Program-independent APIs comprise a set of fixed functions that are independent of the P4 program. Therefore, changes in the P4 programs do not affect those APIs. </div> <br>
<div style="text-align: justify;">
The control plane that manages the data plane tables and externs can be remote or local on the device. Remote control planes invoke API calls through Remote Procedure Calls (RPCs) while relying on asynchronous message frameworks such as Thrift [<a href="#References">1</a>] and gRPC [<a href="#References">2</a>]. Such frameworks use protocol buffers (protobuf) [<a href="#References">3</a>] to define service API and message, and HTTP/2.0 and TLS for transport. On the other hand, a local control plane runs on the Central Processing Unit (CPU) of the device and invokes API calls locally. It is implemented by the driver of the device. </div> <br>

Pipelines are created and managed through Command Line Interface (CLI) [<a href="#References">4</a>]. Figure 1 shows the runtime environment used in this lab series to control the P4 target (P4-DPDK pipeline). The control plane uses the ```DPDK pipeline CLI``` tool to interact with the data plane. The ```DPDK pipeline CLI``` includes a program-independent CLI and a TCP Client (telnet) which connects to the program-independent control server residing on the DPDK pipeline. All the application objects are created and managed through CLI commands. The CLI can also be used to update the pipeline tables or poll the pipeline statistics [<a href="#References">5</a>].

<figure style="text-align: center;">
  <img src="./labs_files/lab6/figs/intro_01.png" width="300" style="display: block; margin: 0 auto;">
  <figcaption>Figure 1. Runtime management of a P4 target (DPDK).</figcaption>
</figure>

# Step 1:  Configuring the environment

Before running this notebook, you will need to configure your environment using the [Configure Environment](../../../configure_and_validate.ipynb) notebook. Please stop here, open and run that notebook, then return to this notebook.

If you are using the FABRIC JupyterHub many of the environment variables will be automatically configured for you.  You will still need to set your bastion username, upload your bastion private key, and set the path to where you put your bastion private key. Your bastion username and private key should already be in your possession.  

If you are using the FABRIC API outside of the JupyterHub you will need to configure all of the environment variables. Defaults below will be correct in many situations but you will need to confirm your configuration.  If you have questions about this configuration, please contact the FABRIC admins using the [FABRIC User Forum](https://learn.fabric-testbed.net/forums/) 

More information about accessing your experiments through the FABRIC bastion hosts can be found [here](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/).

# Step 2: Importing the FABlib library

In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()

# Step 3: Creating the experiment slice

The following creates a node with basic compute and networking capabilities. You build a slice by creating a new slice and adding resources to the slice. After you build the slice, you must submit a request for the slice to be instantiated.   

### Step 3.1: Creating a slice
The code below creates a new slice with the name "P4DPDK_lab6_vnic"

In [2]:
slice = fablib.new_slice(name="P4DPDK_lab6_vnic")

### Step 3.2: Defining the sites
The code below requests a random site from FABRIC based on the condition that the following resources are available:

<ul>
    <li> 4 CPU cores</li>
    <li> 8GB RAM </li>
    <li> 20GB disc size
</ul>

In [3]:
site1= fablib.get_random_sites(count=1, filter_function=lambda x: x['cores_available'] > 4 and x['ram_available'] > 8 and x['disk_available'] > 20)[0]

print (f'The selected site is {site1}')

The selected site is TOKY


### Step 3.3: Creating the nodes
The code below creates one node (server1) which uses the following:
<ul>
    <li> 4 CPU cores</li>
    <li> 8GB RAM </li>
    <li> 20GB disc size </li>
    <li> Image: Ubuntu 20.04
</ul>

server1 will be created in site1

In [4]:
server1 = slice.add_node(name="server1", 
                      site=site1, 
                      cores=4, 
                      ram=8, 
                      disk=20, 
                      image='default_ubuntu_20')

### Step 3.4: Submitting the slice
The code below submits the slice. 
By default, the submit function will block until the node is ready and will display the progress of your slice being built.

In [5]:
slice.submit();


Retry: 16, Time: 406 sec


ID,74535569-1007-4ddf-aade-7ee08b408afd
Name,P4DPDK_lab6_vnic
Lease Expiration (UTC),2024-09-06 02:03:40 +0000
Lease Start (UTC),2024-09-05 02:03:40 +0000
Project ID,8eaa3ec2-65e7-49a3-8c09-e1761141a6ad
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
288b38d7-2c60-414c-8cc6-9a0a119be261,server1,4,8,100,default_ubuntu_20,qcow2,toky-w3.fabric-testbed.net,TOKY,ubuntu,133.69.160.106,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@133.69.160.106,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


# Step 4: Installing the required packages
In this step, we will install the required packages to run the lab. Specifically, we will install the DPDK library, the P4 compiler (p4c), and all needed dependencies.

### Step 4.1 Getting the server node
The command below gets the fablib node that server1 is associated with.

In [6]:
server1 = slice.get_node(name="server1")

### Step 4.2 NAT64 setup
The code below checks if an IPv6 address is available to set up NAT64. We will upload the script [scripts/nat64.sh](./scripts/nat64.sh) to the all servers and execute it

In [7]:
from ipaddress import ip_address, IPv6Address

if type(ip_address(server1.get_management_ip())) is IPv6Address:
    server1.upload_file('scripts/nat64.sh', 'nat64.sh')
    stdout, stderr = server1.execute(f'chmod +x nat64.sh && ./nat64.sh', quiet=True)

### Step 4.3 Installing dependencies
The code below installs packages that are prerequisites to the upcoming installations and needed to run the lab experiments

In [8]:
stdout, stderr = server1.execute(f'sudo apt-get update', quiet = True)
stdout, stderr = server1.execute(f'sudo apt-get install -y build-essential python3-pip python3-pyelftools libnuma-dev pkg-config net-tools syslinux-utils', quiet = True)
stdout, stderr = server1.execute(f'sudo pip3 install meson ninja scapy', quiet = True)

### Step 4.4 Installing DPDK
The code below downloads, builds, and installs DPDK on all servers. In this lab, we are building a modified version of DPDK in which we enabled logs in the terminal to have a better understanding of the behavior of the built pipeline. Note that printing out logs on the terminal degrades performance. Therefore, with applications where high performance is needed, logging should be disabled.  

In [9]:
stdout, stderr = server1.execute(f'git clone http://dpdk.org/git/dpdk', quiet = True)
stdout, stderr = server1.execute(f'cd dpdk/lib/pipeline/ &&  sudo rm rte_swx_pipeline.c && sudo rm rte_swx_pipeline_internal.h', quiet = True)
server1.upload_file('scripts/rte_swx_pipeline.c','/home/ubuntu/dpdk/lib/pipeline/rte_swx_pipeline.c')
server1.upload_file('scripts/rte_swx_pipeline_internal.h','/home/ubuntu/dpdk/lib/pipeline/rte_swx_pipeline_internal.h')
stdout, stderr = server1.execute(f'cd dpdk &&  sudo meson build && cd build && sudo ninja && sudo ninja install && sudo ldconfig', quiet = True)
stdout, stderr = server1.execute(f'cd dpdk/examples/pipeline && sudo make', quiet=True)

### Step 4.5 Install p4c
The code below downloads and installs the p4c compiler needed to compile the p4 code into a DPDK pipeline

In [10]:
stdout, stderr = server1.execute('git clone https://github.com/CILab-USC/p4c.git', quiet = True)
stdout, stderr = server1.execute('sudo apt-get install -y cmake g++ git automake libtool libgc-dev bison flex libfl-dev libboost-dev libboost-iostreams-dev libboost-graph-dev llvm pkg-config python3 python3-pip tcpdump', quiet = True)
stdout, stderr = server1.execute('cd p4c && pip3 install --user -r requirements.txt && mkdir build && cd build && cmake .. && make -j4  && sudo make install', quiet = True)

# Step 5: Defining match-action table rules
This section demonstrates how to define the match-action table rules in text files. The rules will be added to the tables at runtime.

## Step 5.1: Inspecting the control.p4 file

Click on [control.p4](./labs_files/lab6/control.p4) to open the code in the editor.

<img src="./labs_files/lab6/figs/5_01.png" width="750px"><br>

As we inspect the control.p4 file, we can see the defined table names and actions which will be needed to specify the rules and upload them to the tables. The first table is the forwarding_exact table and it takes the forward_exact action only. The second table defined is the forwarding_lpm table and it takes two actions, the forward_lpm and drop.

## Step 5.2: Converting IP to hexadecimal

Now, we will add rules to the text file. The first rule is to forward the packet to host 3 if the destination IPv4 address matches the key. The rule matches on the destination IP address of the packets. The IP address of host 3 is 192.168.30.1. To add it to the rule it must be represented in hexadecimal format.

All parameters to be included in the rules have  to be represented in hexadecimal notation. Type the following command in the terminal to convert an IPv4 address to hexadecimal. 

In [23]:
stdout, stderr = server1.execute('gethostip -x 192.168.30.1')

C0A81E01


The ```gethostip``` command takes an IP address and converts it to a different format. The ```-x``` flag is to specify that we want the IP address to be converted to hexadecimal. 192.168.30.1 is the IP address of host 3. As a result, the IP of host 3 is C0A81E01 in hexadecimal.

## Step 5.3: Add exact matching rules

Click on [rules_exact.txt](./labs_files/lab6/rules_exact.txt) to open the code in the editor.

Now, we will fill the rules_exact.txt file with the rule that will be added to the table for exact matching. Type the following code in the rules_exact.txt file.

    match 0xC0A81E01 action forward_exact dstAddr 0x3 port_id 0x2

<img src="./labs_files/lab6/figs/5_03.png" width="800px"><br>

Defining a rule requires 3 parameters:

•	```match <key>```: The key based on which a match is found or not. The key is 192.168.30.1 (0xC0A81e01 in hexadecimal) which is the IP address of host 3. <br>
•	```action <action name>```: The action to be executed if there is a match. If a match is found the action to be executed for both entries is the forward_exact action declared in the control.p4 code. <br>
•	The last item is the action data In this case, the action data are the destination MAC address and egress port ID. Therefore, if a match is found with this entry, the packet is forwarded to host h3 through port 2 (0x2 in hexadecimal), and the destination MAC address is 00:00:00:00:00:03 (0x3 in hexadecimal).


<hr>

Save the changes by pressing ```Ctrl+s```.

## Step 5.4: Converting IP to hexadecimal

Now, we will add two rules to the text file that correspond to the table applying the Longest Prefix Matching (LPM) algorithm. In LPM rules, the network IP address and the subnet must be defined.

Type the following commands in the terminal to convert an IPv4 address to hexadecimal. 

In [24]:
stdout, stderr = server1.execute('gethostip -x 192.168.10.0')

C0A80A00


The IP address 192.168.10.0 is represented as C0A80A00 in hexadecimal format.

In [25]:
stdout, stderr = server1.execute('gethostip -x 192.168.20.0')

C0A81400


The IP address 192.168.20.0 is represented as C0A81400 in hexadecimal format.

In [26]:
stdout, stderr = server1.execute('gethostip -x 255.255.255.0')

FFFFFF00


The /24 subnet indicates that the first 24 bits of the IP address are used for the network portion. The remaining 8 bits (32 - 24 = 8) are available for host addresses within that network. Therefore, the subnet mask 255.255.255.0 means that the first 24 bits are set to 1 (representing the network), and the last 8 bits are set to 0 (available for hosts). The subnet 255.255.255.0 is represented as FFFFFF00 in hexadecimal format.

## Step 5.5: Add LMP rules

Click on [rules_lpm.txt](./labs_files/lab6/rules_lpm.txt) to open the code in the editor.

Now, we will fill the rules_exact.txt file with the rules that will be added to the table for exact matching. Type the following code in the rules_lpm.txt file.

    match 0xC0A80A00/0xffffff00 action forward_lpm dstAddr 0x1 port_id 0x0
    match 0xC0A81400/0xffffff00 action forward_lpm dstAddr 0x2 port_id 0x1

<img src="./labs_files/lab6/figs/5_05.png" width="800px"><br>

Defining a rule requires 3 parameters:

•	```match <key>```: The key based on which a match is found or not. In the first entry, the key is 192.168.10.0/24 (0xC0A80A00/ffffff00 in hexadecimal) which is the IP address and netmask of host 1. The match key of the second entry is 192.168.20.0/24 (0xC0A81400/ffffff00 in hexadecimal) which is the IP address and netmask of host 2. <br>
•	```action <action name>```: The action to be executed if there is a match. If a match is found the action to be executed for both entries is the forward_lpm action declared in the control.p4 code which was written earlier in this lab. <br>
•	The last item is the action data. In this case, the action data are the destination MAC address and egress port ID. Therefore, if a match is found with the first entry, the packet is forwarded to host 1 through port 0 (0x0 in hexadecimal) and the destination MAC address is 00:00:00:00:00:01 (0x1 in hexadecimal). Whereas, if a match is found with the second entry, the packet is forwarded to host 2 through port 1 (0x1 in hexadecimal) and the destination MAC address is 00:00:00:00:00:02 (0x2 in hexadecimal).

<hr>

Save the changes by pressing ```Ctrl+s```.

## Step 5.6: Compiling the P4 code
To upload all needed P4 codes and compile main.p4, issue the following command.

In [27]:
server1.upload_file('labs_files/lab6/headers.p4','headers.p4')
server1.upload_file('labs_files/lab6/parser.p4','parser.p4')
server1.upload_file('labs_files/lab6/precontrol.p4','precontrol.p4')
server1.upload_file('labs_files/lab6/control.p4','control.p4')
server1.upload_file('labs_files/lab6/deparser.p4','deparser.p4')
server1.upload_file('labs_files/lab6/main.p4','main.p4')
stdout, stderr = server1.execute(f'sudo p4c-dpdk --arch=pna main.p4 -o lab6.spec')
stdout, stderr = server1.execute(f'ls')

control.p4
deparser.p4
dpdk
headers.p4
host_tune.sh
lab6.spec
main.p4
p4c
parser.p4
precontrol.p4


The command above invokes the ```p4c-dpdk``` compiler to compile the ```main.p4``` program and generates the ```lab6.spec``` file which is a specification file needed to run the pipeline.

# Step 6: Running the P4-DPDK pipeline and the lab topology
This section shows the steps required to run the P4-DPDK along with building the lab topology. In this lab, the procedure is automated.

## Step 6.1: Uploading required scripts
The code below uploads to server1 the CLI and I/O scripts, along with other scripts needed to automate the process of running the pipeline and building the topology.

We will also upload the rules.txt file to load and add rules to the forwarding table. The rules.txt file contains rules for exact matching.

In [28]:
server1.upload_file('labs_files/lab6/lab6.cli','lab6.cli')
server1.upload_file('labs_files/lab6/ethdev.io','ethdev.io')
server1.upload_file('labs_files/lab6/run_pipeline.sh','run_pipeline.sh')
server1.upload_file('labs_files/lab6/set_topology.sh','set_topology.sh')
server1.upload_file('labs_files/lab6/sender.py','sender.py')
server1.upload_file('labs_files/lab6/rules_exact.txt','rules_exact.txt')
server1.upload_file('labs_files/lab6/rules_lpm.txt','rules_lpm.txt')
stdout, stderr = server1.execute(f'chmod +x run_pipeline.sh')
stdout, stderr = server1.execute(f'chmod +x set_topology.sh')
stdout, stderr = server1.execute(f'chmod +x sender.py')

## Step 6.2: Opening a terminal
Launch a new terminal by opening a new tab and then select "terminal".

<img src="./labs_files/lab6/figs/6_02_terminal.gif" width="1000px"><br>

Copy the output of the command below and paste it into the terminal to enter server1.

In [22]:
server1.get_ssh_command()

'ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@133.69.160.106'

## Step 6.3: Running the P4-DPDK pipeline
Build and run the pipeline by typing the following command:

    sudo ./run_pipeline.sh
    
<img src="./labs_files/lab6/figs/6_03.png" width="750px"><br>

The ```run_pipeline.sh``` script is a shell script that automates the process of running the P4-DPDK pipeline. In this lab, part of the pipeline output is hidden to display only the relevant logs.

## Step 6.4: Building the lab topology
The code below uploads to server1 the CLI and I/O scripts, along with other scripts needed to automate the process of running the pipeline and building the topology.

In [35]:
stdout, stderr = server1.execute(f'sudo ./set_topology.sh')

The set_topology.sh script is a shell script that automates the process of building the lab topology. Two namespaces are built and configured in this step with a virtual device linked to each as shown in the figure below.

<img src="./labs_files/lab6/figs/6_04_ip.png" width="650px"><br>

# Step 7: Running commands at runtime in the DPDK pipeline CLI

### Step 7.1: Opening a new terminal
Launch a new terminal by opening a new tab and then select "terminal".

<img src="./labs_files/lab6/figs/7_01_terminal.gif" width="1000px"><br>

Copy the output of the command below and paste it into the terminal to enter server1.

### Step 7.2: Access DPDK pipeline CLI (Command Line Interface)
To start the DPDK pipeline CLI by typing the following command.

    telnet 0.0.0.0 8086

<img src="./labs_files/lab6/figs/7_02.png" width="700px"><br>

Defining a rule requires 3 parameters:

The ```telnet``` command is followed by the IP address of the server (0.0.0.0) and the port number (8086).


### Step 7.3: Inspect pipeline interfaces
In the pipeline CLI, inspect the interface configuration by typing the following command.

    ethdev show

<img src="./labs_files/lab6/figs/7_03.png" width="700px"><br>

Defining a rule requires 3 parameters:

We can see that three interfaces are displayed (highlighted in the grey boxes); net_tap0, net_tap1, and net_tap2.

As we inspect the output for each ethernet device in the figure above, we can see in the first line, that ```ether``` is the MAC addresses assigned to the interfaces while running the pipeline along with ```rxqueues``` and ```txqueues```, the number of receiving and transmitting queues as assigned in the CLI script while listing the ethernet devices. The second line shows the port number ```port#``` as specified in the I/O file. The remaining lines of the output show the received packet count ```RX packets``` and the transmitted packet count ```TX packets``` with the corresponding total byte-count ```bytes``` and the number of dropped packets ```misses``` for each.


### Step 7.4: Inspect pipeline stats
In the pipeline CLI, inspect the pipeline statistics by typing the following command.

    pipeline PIPELINE0 stats

<img src="./labs_files/lab6/figs/7_04.png" width="700px"><br>

The pipeline stats function shows statistical data of the specified pipeline (PIPELINE0). This pipeline has three ```Input ports``` highlighted in the first grey box and three ```Output ports``` highlighted in the second grey box. Also, each port is associated with counters that provide information about the number of packets and the total byte count. 

The ```Tables``` section indicates that there are two tables defined, the ```forwarding_exact``` which applies exact matching, and the ```forwarding_lpm``` which applies the LPM algorithm. Each table is followed by the count of packets hit or miss along with the number of packets that underwent each action. Note that at the bottom, there are no learner tables defined because the ```add_on_miss``` feature is not enabled in this example.



### Step 7.5: Adding rules to exact matching table
Add the following code inside the pipeline CLI to add rules to the forwarding_exact table.

    pipeline PIPELINE0 table forwarding_exact add rules_exact.txt

<img src="./labs_files/lab6/figs/7_05.png" width="700px"><br>

To add rules to a table the pipeline table add function is used. This command takes three parameters:

•	```pipeline```: The table is processed in the specified DPDK pipeline (PIPELINE0). <br>
•	```table```: These rules are added to the specified table (forwarding_exact). Note that the name of the table should match the name used in the table declaration in the control.p4 code. <br>
•	```add```: This parameter takes the path to the file containing the rules to be added to the table (rules_exact.txt).



### Step 7.6: Adding rules to LPM table
Add the following code inside the pipeline CLI to add rules to the forwarding_lpm table.

    pipeline PIPELINE0 table forwarding_lpm add rules_lpm.txt

<img src="./labs_files/lab6/figs/7_06.png" width="700px"><br>

### Step 7.7: Commit changes in the pipeline
To confirm changes made to the pipeline, type the following code inside the pipeline CLI.

    pipeline PIPELINE0 commit

<img src="./labs_files/lab6/figs/7_07.png" width="700px"><br>

The ```pipeline commit``` function applies the changes made to the designated pipeline (PIPELINE0).

# Step 8: Testing the application

### Step 8.1: Sending packets from h1 to h2
Now, we will send a packet from h1 to h2 over the destination IP address 192.168.20.2 by running the provided Python script sender.py.

In [36]:
stdout, stderr = server1.execute(f'sudo ip netns exec h1 python3 sender.py -s h1 -d 192.168.20.2')

. WARNING: Interface lo: no address assigned
 Sender: h1
Destination IP: 192.168.20.2

Sent 1 packets.


Running the Python script requires two parameters:

•	```-s```: Sender (h1 or h2 or h3) <br>
•	```-d```: Destination IP Address 

None of the rules uploaded include matching on the IP address 192.168.20.2. Therefore, the LPM rules are applied to match the IP address to the longest prefix which is in this case 192.168.20.0.

### Step 8.2: Inspect the pipeline terminal
Observe the DPDK logs at the bottom of the terminal. These logs correspond to the packet processing function executed in the .spec file generated when the P4 code is compiled. 

<img src="./labs_files/lab6/figs/8_02.png" width="700px"><br>

The log highlighted in the first grey box corresponds to the table lookup. An exact match was not found in table 0 which corresponds to the forwarding_exact table. In this case, a table lookup will be performed in the second table named forwarding_lmp with a table ID of 1. A match was found in the second table. Therefore, the forward_lpm (action 2) is executed.

The “tx 1 pkt to port 1” log in the second grey box indicates that the forwarding action is properly executed by sending one packet to port 1 which corresponds to host 2.

For a more readable output press enter in the terminal a few times (five times) to provide space for the next logs.

### Step 8.3: Inspect the pipeline stats
In the pipeline CLI, inspect the pipeline statistics by typing the following command.

    pipeline PIPELINE0 stats
    
<img src="./labs_files/lab6/figs/8_03.png" width="700px"><br>

In the ```Tables``` section, under the stats of the table ```forwarding_exact```, we can see in the grey box that 1 packet was processed and no match was found. Therefore, no action was executed and now the packet will have to be looked up within the rules in the second table. Under the stats of the table ```forwarding_lpm```, a match was found and the action ```forward_lpm``` was executed. 


# Step 9: Deleting rules at runtime

### Step 9.1: Removing exact match-action table rules
Enter the following command in the pipeline CLI to delete rules from a table.

    pipeline PIPELINE0 table forwarding_exact delete rules_exact.txt
    
<img src="./labs_files/lab6/figs/9_01.png" width="700px"><br>

To delete rules from a table, the pipeline table delete function is used. This command takes three parameters:

•	```pipeline```: The table is processed in the specified DPDK pipeline (PIPELINE0). <br>
•	```table```: These rules are deleted from the specified table (forwarding_exact). Note that the name of the table should match the name used in the table declaration in the control.p4 code. <br>
•	```delete```: This parameter takes the path to the file containing the rules to be deleted from the table (rules_exact.txt).



### Step 9.2: Commint changes in the pipeline
To confirm changes made to the pipeline, type the following code inside the pipeline CLI.

    pipeline PIPELINE0 commit

<img src="./labs_files/lab6/figs/9_02.png" width="700px"><br>

The ```pipeline commit``` function applies the changes made to the designated pipeline (PIPELINE0).


### Step 9.3: Sending packets from h1 to h3
Now, we will send a packet from h1 to h3 over the destination IP address 192.168.30.1 by running the provided Python script sender.py.

In [37]:
stdout, stderr = server1.execute(f'sudo ip netns exec h1 python3 sender.py -s h1 -d 192.168.30.1')

. WARNING: Interface lo: no address assigned
 Sender: h1
Destination IP: 192.168.30.1

Sent 1 packets.


### Step 9.4: Inspect the pipeline terminal
Observe the DPDK logs at the bottom of the terminal. These logs correspond to the packet processing function executed in the .spec file generated when the P4 code is compiled. 

<img src="./labs_files/lab6/figs/9_04.png" width="700px"><br>

The log highlighted in the first grey box corresponds to the table lookup. An exact match was not found in table 0 which corresponds to the forwarding_exact table. In this case, a table lookup will be performed in the second table named forwarding_lmp with a table ID of 1. A match was not found in the second table. Therefore, the default action (action 3) is executed and the packet is dropped. The log in the second grey box indicates that one packet has been dropped.

### Step 9.5: Inspect the pipeline stats
In the pipeline CLI, inspect the pipeline statistics by typing the following command.

    pipeline PIPELINE0 stats
    
<img src="./labs_files/lab6/figs/9_05.png" width="700px"><br>

In the ```Output ports``` section, we can see in the grey box that the pipeline dropped the 1 packet. 

In the ```Tables``` section, under the stats of the table ```forwarding_exact```, we can see in the grey box that the packet was processed, and no match was found. Therefore, no action was executed and now the packet will have to be looked up within the rules in the second table. Under the stats of the table ```forwarding_lpm```, a match was not found, and the action drop was executed. This is because the rule that applies exact matching of the destination address of host 3 is deleted.

### Step 9.6: End the telnet session
Close the pipeline CLI and the telnet session by pressing ```ctrl+]``` and then type the ```quit``` command.
    
<img src="./labs_files/lab6/figs/9_06.png" width="700px"><br>

### Step 9.7: Stopping the DPDK pipeline
Stop the DPDK pipeline by pressing ```ctrl+c``` in the terminal running the pipeline.

<img src="./labs_files/lab6/figs/9_07.png" width="700px"><br>

# Step 10: Delete the slice

This concludes Lab 6. Please delete your slice when you are done with your experiment.

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()
slice = fablib.get_slice(name="P4DPDK_lab6_vnic")
slice.delete()

# References

1.	Apache. “Apache Thrift.” [Online]. Available: https://thrift.apache.org/.
2.	Google. “gRPC.” [Online]. Available: https://grpc.io/.
3.	Google. “Protocol Buffers.” [Online]. Available:
https://developers.google.com/protocol-buffers.
4.	DPDK, “Pipeline Application”, [Online]. Available: https://doc.dpdk.org/guides/sample_app_ug/pipeline.html.
5.	DPDK, “Internet Protocol (IP) Pipeline Application”. [Online]. Available: https://doc.dpdk.org/guides-18.08/sample_app_ug/ip_pipeline.html.